Task 1

In [1]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.0 MB/s eta 0:00:00


In [5]:
# -------------------------------
# 📌 Block 1: Import libraries & load data
# -------------------------------
import pandas as pd
from rapidfuzz import fuzz, process

# Load the resolved queries
resolved_df = pd.read_csv("resolved_queries (1).csv")

# Load the new queries
new_df = pd.read_csv("new_queries (1).csv")

print("Resolved queries:", resolved_df.shape)
print("New queries:", new_df.shape)
print(resolved_df.head())
print(new_df.head())

Resolved queries: (5, 2)
New queries: (20, 2)
   Query_ID                    Pre_Resolved_Query
0         1     Unable to connect to the internet
1         2        Payment failed during checkout
2         3     App crashes when opening settings
3         4   Forgot password and unable to reset
4         5  Unable to upload files to the server
                             Variation_Query  Matches_With_Query_ID
0           Unabel to conect to the internet                      1
1                  Can’t connect to internet                      1
2                        Intenet not working                      1
3               Payment failed while chekout                      2
4  Payment did not go through during chckout                      2


In [6]:
# -------------------------------
# 📌 Block 2: Define fuzzy matching function
# -------------------------------
def fuzzy_match(query, choices, scorer=fuzz.token_sort_ratio, threshold=70):
    """
    Match a query string against a list of choices using RapidFuzz.
    Returns the best match and its score if above threshold, else None.
    """
    match, score, idx = process.extractOne(query, choices, scorer=scorer)
    if score >= threshold:
        return match, score
    return None, None


In [10]:
# -------------------------------
# 📌 Block 3: Apply fuzzy matching to new queries
# -------------------------------
results = []

# Choices come from the resolved queries column
resolved_queries = resolved_df["Pre_Resolved_Query"].tolist()

for i, row in new_df.iterrows():
    query = row["Variation_Query"]  # <-- use Variation_Query instead of Query
    match, score = fuzzy_match(query, resolved_queries, scorer=fuzz.token_sort_ratio, threshold=70)

    if match:
        matched_id = resolved_df.loc[resolved_df["Pre_Resolved_Query"] == match, "Query_ID"].values[0]
        results.append((query, match, matched_id, score))
    else:
        results.append((query, None, None, None))

# Convert to DataFrame
matches_df = pd.DataFrame(results, columns=["New_Query", "Matched_Query", "Matched_Query_ID", "Score"])
# Attach fuzzy predictions back to new_df
new_df["Fuzzy_Pred"] = matches_df["Matched_Query_ID"].values
new_df["Fuzzy_Score"] = matches_df["Score"].values

print(matches_df.head(10))


                                   New_Query  \
0           Unabel to conect to the internet   
1                  Can’t connect to internet   
2                        Intenet not working   
3               Payment failed while chekout   
4  Payment did not go through during chckout   
5                 Payment issue at check out   
6   Application crashes when opening setings   
7           App crash when going to settings   
8           Settings cause the app to chrash   
9              Forgot passwrd and cant reset   

                         Matched_Query  Matched_Query_ID      Score  
0    Unable to connect to the internet               1.0  95.384615  
1    Unable to connect to the internet               1.0  72.413793  
2                                 None               NaN        NaN  
3       Payment failed during checkout               2.0  75.862069  
4                                 None               NaN        NaN  
5                                 None             

In [12]:
# ====================================================
# Block 4: Evaluate Fuzzy Matching
# ====================================================
from sklearn.metrics import accuracy_score, f1_score

# Replace NaN with -1 for unmatched queries
new_df["Fuzzy_Pred"] = new_df["Fuzzy_Pred"].fillna(-1)

true_ids = new_df["Matches_With_Query_ID"]
fuzzy_preds = new_df["Fuzzy_Pred"]

acc = accuracy_score(true_ids, fuzzy_preds)
f1 = f1_score(true_ids, fuzzy_preds, average="macro")

print(f"Fuzzy Matching (token_sort_ratio) -> Accuracy: {acc:.3f}, F1: {f1:.3f}")


Fuzzy Matching (token_sort_ratio) -> Accuracy: 0.450, F1: 0.511


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Optional: simple preprocessing
def preprocess(text):
    return text.lower().strip()

resolved_df["processed_query"] = resolved_df["Pre_Resolved_Query"].apply(preprocess)
new_df["processed_query"] = new_df["Variation_Query"].apply(preprocess)

# Vectorize queries
vectorizer = TfidfVectorizer()
tfidf_matrix_resolved = vectorizer.fit_transform(resolved_df["processed_query"])
tfidf_matrix_new = vectorizer.transform(new_df["processed_query"])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix_new, tfidf_matrix_resolved)

# Pick best match for each new query
pred_ids_tfidf = []
for i in range(len(new_df)):
    best_idx = np.argmax(cosine_sim[i])
    pred_ids_tfidf.append(resolved_df.iloc[best_idx]["Query_ID"])

new_df["TFIDF_Pred"] = pred_ids_tfidf

print(new_df[["Variation_Query", "Matches_With_Query_ID", "TFIDF_Pred"]].head(10))


                             Variation_Query  Matches_With_Query_ID  \
0           Unabel to conect to the internet                      1   
1                  Can’t connect to internet                      1   
2                        Intenet not working                      1   
3               Payment failed while chekout                      2   
4  Payment did not go through during chckout                      2   
5                 Payment issue at check out                      2   
6   Application crashes when opening setings                      3   
7           App crash when going to settings                      3   
8           Settings cause the app to chrash                      3   
9              Forgot passwrd and cant reset                      4   

   TFIDF_Pred  
0           1  
1           1  
2           1  
3           2  
4           2  
5           2  
6           3  
7           3  
8           3  
9           4  


In [15]:
# ====================================================
# Block 6: Evaluate TF-IDF Matching
# ====================================================
tfidf_preds = new_df["TFIDF_Pred"]

acc_tfidf = accuracy_score(true_ids, tfidf_preds)
f1_tfidf = f1_score(true_ids, tfidf_preds, average="macro")

print(f"TF-IDF Cosine -> Accuracy: {acc_tfidf:.3f}, F1: {f1_tfidf:.3f}")


TF-IDF Cosine -> Accuracy: 1.000, F1: 1.000


In [16]:
# ====================================================
# Block 7: Save Results
# ====================================================
output_path = Path("task1_results.csv")
new_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path.absolute()}")


Results saved to /content/task1_results.csv


Task 2

In [4]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.7 MB/s eta 0:00:00


In [22]:
!pip install rapidfuzz pandas jellyfish

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 6.9 MB/s eta 0:00:00


In [23]:
import pandas as pd
from rapidfuzz import fuzz, process
import jellyfish

# --- Load CSVs ---
base = pd.read_csv("base_names.csv")
variations = pd.read_csv("name_variations.csv")

print("Base names columns:", base.columns.tolist())
print("Name variations columns:", variations.columns.tolist())

# --- Rename columns for consistency ---
# Assuming first col is ID and second is name
base = base.rename(columns={base.columns[0]: "id", base.columns[1]: "name"})
variations = variations.rename(columns={variations.columns[0]: "id", variations.columns[1]: "name"})

print("Base names shape:", base.shape)
print("Variations shape:", variations.shape)

Base names columns: ['Base_Name_ID', 'Base_Name']
Name variations columns: ['Variation', 'Matches_With_Base_Name']
Base names shape: (20, 2)
Variations shape: (100, 2)


In [24]:
# --- Preprocess names ---
def clean_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower().strip()
    name = name.replace(",", " ")        # handle "Smith, John"
    name = " ".join(name.split())        # remove extra spaces
    return name

base["name_clean"] = base["name"].apply(clean_name)
variations["name_clean"] = variations["name"].apply(clean_name)

# --- Fuzzy match names ---
def match_name(name, choices, threshold=90):
    match = process.extractOne(name, choices, scorer=fuzz.token_set_ratio, score_cutoff=threshold)
    return match

name_matches = []
for n in variations["name_clean"]:
    match = match_name(n, base["name_clean"].tolist())
    name_matches.append(match)

variations["best_match"] = [m[0] if m else None for m in name_matches]
variations["match_score"] = [m[1] if m else None for m in name_matches]

print("\n=== Fuzzy Name Matching Results (Top 5) ===")
print(variations[["name", "best_match", "match_score"]].head())



=== Fuzzy Name Matching Results (Top 5) ===
           name    best_match  match_score
0   Thomas King   thomas king        100.0
1   Thomas King   thomas king        100.0
2  Maria Garcia  maria garcia        100.0
3    Mary Lewis    mary lewis        100.0
4  Nancy Wright  nancy wright        100.0


In [25]:
jw_scores = []
for i, row in variations.iterrows():
    if row["best_match"]:
        jw = jellyfish.jaro_winkler_similarity(row["name_clean"], row["best_match"])
        jw_scores.append(jw)
    else:
        jw_scores.append(None)

variations["jaro_winkler_score"] = jw_scores

# --- Save results ---
variations.to_csv("task2_name_matches.csv", index=False)
print("\n✅ Task 2 results saved as task2_name_matches.csv")


✅ Task 2 results saved as task2_name_matches.csv
